<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality and Cleanliness

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Homicides Colombia ([datos.gov.co](datos.gov.co))
  
Last update: September, 2023

In [ ]:
!pip install pylev

In [ ]:
import re
from random import randint
from datetime import datetime
from difflib import SequenceMatcher

import numpy as np
import pandas as pd

import pylev

In [ ]:
pd.set_option("display.max_columns", None)

## 1. Loading the data

In [ ]:
homicides_df = pd.read_csv("./data/homicides.csv")

In [ ]:
homicides_df.shape

(12400, 22)

In [ ]:
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [ ]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
9387,10/08/2021 12:00:00 AM,ANTIOQUIA,URRAO,Jueves,19:05,LA GUAYABALA,RURAL,FINCAS Y SIMILARES,ARMA DE FUEGO,A PIE,A PIE,20.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,5847000.0,1995.0,78-485,bitipn8237@gmail.com
5585,06/15/2021 12:00:00 AM,RISARALDA,PEREIRA (CT),Lunes,21:20,GUAYACANES COM. SAN JOAQUIN,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,66001000.0,1994.0,50-997,dmfgbl1450@gmail.com
7980,08/26/2021 12:00:00 AM,ANTIOQUIA,CALDAS,Mirrcoles,15:24,VIA ANGELOPOLIS,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,ANALFABETA,COLOMBIA,5129000.0,1985.0,53-477,djaqib5001@gmail.com
9667,10/17/2021 12:00:00 AM,VALLE,CALI (CT),Sábado,20:35,ALFONSO B. ARAGON E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,20.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,50-664,ffbhef2243@gmail.com
6280,07/05/2021 12:00:00 AM,ATLÁNTICO,BARRANQUILLA (CT),Domingo,22:30,LOS OLIVOS I,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,PASAJERO MOTOCICLETA,28.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1987.0,93-259,correo967@unidatos.edu.co


## 2. Working with datetimes

In [ ]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [ ]:
# Applying the validation to all values in the column
homicides_df["FECHA"].apply(dateparse)

# IT IS EXPECTED TO HAVE AN ERROR BECAUSE SOME VALUES DOESN'T FIT THE FORMAT

ValueError: time data '13/12/2021 12:00:00 AM' does not match format '%m/%d/%Y %H:%M:%S %p'

In [ ]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [ ]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
486,13/12/2021 12:00:00 AM,VALLE,CALI (CT),kunes,23:00,MOJICA E15,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,89.0,80-330,lujhdf9132@gmail.com
695,30/01/2021 12:00:00 AM,BOLÍVAR,CARTAGENA (CT),Lunes,5:30,REP. DEL LIBANO,URBANA,VIAS PUBLICAS,ARMA BLANCA,NO REPORTADO,A PIE,25.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,13001000.0,1990.0,12-915,ghumtg4094@unidatos.edu.co
1250,18/05/2021 12:00:00 AM,HUILA,TESALIA,Jueves,19:30,VEREDA PACARNI,RURAL,CASAS DE HABITACION,ARMA DE FUEGO,A PIE,A PIE,34.0,FEMENINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,41797000.0,1981.0,99-095,sdaggf6639@gmail.com
12168,12/25/2021 12:00:00 MM,VALLE,PALMIRA,Viernes,15:30,LA EMILIA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,17.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1998.0,16-362,uschca1775@gmail.com
12399,TOTAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Deleting a row by its index
homicides_df.drop([486, 695, 1250, 12168, 12399], inplace=True)

In [ ]:
# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [ ]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
HORA                         object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [ ]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12395
Name: FECHA, dtype: int64

*The homicide hour is available in a different column!*

In [ ]:
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11])+homicides_df["HORA"]

In [ ]:
# Deleting redundant column
homicides_df.drop(columns=["HORA"], inplace=True)

In [ ]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [ ]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [ ]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [ ]:
homicides_df.sample(5)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
3498,2021-04-16 03:30:00,ANTIOQUIA,MARINILLA,Jueves,MARIA AUXILIADORA,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5440000.0,1989.0,97-218,inkkoh7578@gmail.com
1863,2021-02-26 04:40:00,N. DE SANTANDER,TIBÚ,Jueves,BARRIO BUENOS AIRES,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,54810000.0,1994.0,70-048,pderse6281@gmail.com
3107,2021-04-04 17:48:00,VALLE,CALI (CT),Sábado,SINDICAL E12,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,20.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,86-738,mmbuqq1303@gmail.com
10553,2021-11-14 23:30:00,CAUCA,SANTA ROSA,Sábado,SAN JUAN DE VILLALOBOS,RURAL,CASAS DE HABITACION,ARMA BLANCA,A PIE,A PIE,21.0,FEMENINO,UNION LIBRE,AMA DE CASA,NO REPORTADO,PRIMARIA,COLOMBIA,19701000.0,1994.0,55-228,uetmgu3753@unidatos.edu.co
9724,2021-10-18 13:20:00,VALLE,PRADERA,Dominog,ALTO DEL CASTILLO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,CONDUCTOR MOTOCICLETA,23.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76563000.0,1992.0,60-102,qoijbt6931@unidatos.edu.co


## 2. Fixing categorical column

In [ ]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [ ]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TODO: Replace the values identified as error to a valid value from the dictionary.</span>

<span style="color:red">Hint: You can use the replace() pandas function.</span>

In [ ]:
homicides_df["DEPARTAMENTO"] = homicides_df["DEPARTAMENTO"].replace(to_replace="SAN ANDRÉS",
           value="SAN ANDRÉS Y PROVIDENCIA")

homicides_df["DEPARTAMENTO"] = homicides_df["DEPARTAMENTO"].replace(to_replace="N. DE SANTANDER",
           value="NORTE DE SANTANDER")

In [ ]:
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

Ya no quedan registros cuyo valor este por fuera del dominio de departamentos válidos

## 3. Analyzing potential duplicates

In [ ]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]

In [ ]:
duplicates_by_cedula.shape

(1627, 21)

In [ ]:
duplicates_by_cedula.sort_values("CÉDULA", ascending=True).head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co
10796,2021-11-20 03:00:00,VALLE,CALI (CT),Viernes,POTRERO GRANDE E21,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,21.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1994.0,10-079,iknnoj8430@unidatos.edu.co
2386,2021-03-14 02:00:00,META,FUENTE DE ORO,Sábado,VEREDA PUERTO NUEVO,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA BLANCA,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,50287000.0,1973.0,10-255,afompq7113@unidatos.edu.co
138,2021-01-02 06:20:00,CAUCA,SOTARA,Viernes,CENTRO,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,22.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19760000.0,1993.0,10-255,cobgqs8819@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated.</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [ ]:
homicides_df = homicides_df.drop_duplicates(subset=['CÉDULA'])

In [ ]:
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep=False)]
duplicates_by_cedula.shape

(0, 21)

## 4. Fixing formats

In [ ]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [ ]:
cedula_malformed.shape

(28, 21)

In [ ]:
cedula_malformed.head(6)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co
3409,2021-04-13 23:20:00,CÓRDOBA,SAHAGÚN,Lunes,CORREGIMIENTO DE BAJO GRANDE,URBANA,BILLARES,CONTUNDENTES,A PIE,A PIE,49.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,23660000.0,1966.0,496-18,fdbbeo6751@unidatos.edu.co
3588,2021-04-19 20:00:00,BOLÍVAR,CARTAGENA (CT),Domingo,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,NO REPORTADO,NO REPORTADO,13001000.0,1970.0,188-03,diebuo5651@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [ ]:
homicides_df["CÉDULA"] = homicides_df["CÉDULA"].apply(lambda x: (x.replace('-','')))

In [ ]:
homicides_df["CÉDULA"] = homicides_df["CÉDULA"].str[:2] + "-" + homicides_df["CÉDULA"].str[2:]

Verificamos que no hayan cedulas mal formadas

In [ ]:
homicides_df["CÉDULA"].head(2)

0    42-908
1    15-183
Name: CÉDULA, dtype: object

In [ ]:
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]
print(cedula_malformed.shape)

(0, 21)


Ya no quedan cedulas con valores que no siguen el lineamiento

<span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [ ]:
mail_malformed = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.match(".*\d{4}@(.*\.edu\.co|.*\.com)", x) is None))]
mail_malformed["CORREO"].head(6)

28     correo975@unidatos.edu.co
36    correo4714@colombia.gov.co
47    correo8297@colombia.gov.co
55    correo7285@colombia.gov.co
60     correo870@unidatos.edu.co
63     correo102@unidatos.edu.co
Name: CORREO, dtype: object

In [ ]:
mail_malformed.shape

(2134, 21)

Existen 2134 correos no validos

In [ ]:
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [ ]:
# Calculating the distance between two words using the Levenshtein distance
pylev.levenshtein("sábado", "sabaod")

3

In [ ]:
pylev.levenshtein("sábado", "viernes")

7

In [ ]:
SequenceMatcher(None, "sábado", "sabaod").ratio()

0.6666666666666666

In [ ]:
SequenceMatcher(None, "sábado", "viernes").ratio()

0.15384615384615385

<span style="color:red">How does SequenceMatcher works? How this differ from the Levenshtein distance?</span>

SequenceMatcher implementa el algoritmo LCS (Longest Contiguous Matching Subsequence), cuyo objetivo es encontrar la longitud de la subsecuencia más larga presente en ambas cadenas de texto. El método ratio retorna un score de similitud entre 0 y 1, que se calcula como 2 * M/T, donde M es la cantidad de elementos en comun, y T el número total de elementos en las dos secuencias. Entre más cercano a 1 sea el valor, más parecidas son las secuencias.

<span style="color:red">TODO: Create a function to fix the digitation errors for the "DIA" column.</span>

In [ ]:
dias = ["Lunes","Martes","Miercoles","Jueves","Viernes","Sábado","Domingo"]

In [ ]:
def apply_sequenceMatcher(x):
    diaRetornado = x
    for dia in dias:
        if SequenceMatcher(None, x, dia).ratio() > 0.6:
            diaRetornado = dia
            break
    return diaRetornado

In [ ]:
homicides_df["DIA"] = homicides_df["DIA"].apply(apply_sequenceMatcher)

In [ ]:
homicides_df["DIA"].unique()

array(['Jueves', 'Miercoles', 'Viernes', 'Sábado', 'Domingo', 'Lunes',
       'Martes'], dtype=object)

Los valores de dia de todos los registros corresponden al dominio de valores válido

## 6. Recalculation based on a different column

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
11524,1986.0,29.0
3244,1979.0,36.0
1912,1991.0,24.0
429,1994.0,21.0
5069,1992.0,23.0
1749,2000.0,15.0
12396,1997.0,18.0
1035,1958.0,57.0
2560,1996.0,19.0
7573,1990.0,25.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [ ]:
homicides_df["EDAD"] = datetime.now().date().year - homicides_df["AÑO DE NACIMIENTO"]

In [ ]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].sample(10)

,AÑO DE NACIMIENTO,EDAD
1137,1994.0,29.0
9061,1979.0,44.0
10311,1965.0,58.0
3755,1978.0,45.0
8184,1956.0,67.0
10316,2014.0,9.0
1268,1951.0,72.0
2140,1993.0,30.0
8510,1991.0,32.0
2658,2001.0,22.0


In [ ]:
uncommon_age = homicides_df.loc[(homicides_df["EDAD"] > 100) | (homicides_df["EDAD"] < 0)]

In [ ]:
uncommon_age[["AÑO DE NACIMIENTO","EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
56,75.0,1948.0
93,69.0,1954.0
104,66.0,1957.0
136,85.0,1938.0
178,97.0,1926.0
274,86.0,1937.0
298,75.0,1948.0
416,87.0,1936.0
442,78.0,1945.0
468,96.0,1927.0


In [ ]:
uncommon_age.shape

(223, 21)

In [ ]:
homicides_df.shape

(11559, 21)

Según parece, existen registros en los cuales el año de nacimiento no fue registrado con los 4 digitos correspondientes sino solamente los dos últimos. De los 11559 registros restantes luego del procesamiento realizado en este notebook, eliminando por ejemplo los registros por cédula, se ve que estos registros con año irregular son apenas el 2% de la muestra.

## 7. Conclusion

<span style="color:red">Make a summary of the different data quality problems found on the dataset, the data quality dimension that is related to and the implemented strategy for solving or mitigating that specific problem.</span>

| Atributo           | Dimensión de calidad no cumplida |Problema identificado                                                                                           |Acciones realizadas
|--------------------|--------------------|------------------------------------------------------------------------------------------------|----|
| Fecha              | Consistencia       | No todas las fechas seguían el formato que se supone deberían tener| Se identificaron los registros inconsistentes usando expresiones regulares, y se eliminaron.                                                                      |
| Fecha              | Conformidad, Precisión | No contenia la hora, la hora estaba en otra columna| Se unió con la columna de hora, se eliminó la columna de hora                                                                           |
| Departamento       | Consistencia       | Habían varios registros cuyos valores estaban por fuera del dominio valido| Se encontraron los valores no válidos y se reemplazaron por el correcto.                                                          |
| Cédula             | Consistencia       | Registros duplicados| Se eliminaron los registros duplicados por cedula.                                   |
| Cédula             | Conformidad        | Registros que no seguían el patrón establecido| Se modificaron los registros para que todos siguieran el patrón establecido usando expresiones regulares          |
| Correo             | Conformidad        | Registros que no seguían el patrón establecido| Se identificaron los correos que no seguian el patrón establecido usando expresiones regulares                |
| Día                | Consistencia       | Registros que no están en el dominio de datos establecido| Se usó el método SequenceMatcher para reemplazar los dias no validos con el dia que tuviera una mayor similitud        |
| Año de nacimiento  | Temporalidad       | No corresponde al valor actual| Se calculó el valor actual usando la columna año de nacimiento.                                        |